In [1]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
import sys, pathlib
from pathlib import Path
_THIS_FILE = Path(globals().get("__file__", Path.cwd())).resolve()
PROJECT_ROOT = _THIS_FILE.parent
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *
from utils.eval import *
from utils.dataloader import *

device   = "cuda"
vocab = dataset.vocab
index_to_token = {idx: token for token, idx in vocab.items()}

{'[#C]': 0, '[#N]': 1, '[*H0]': 2, '[=*H0]': 3, '[=Branch]': 4, '[=C]': 5, '[=N]': 6, '[=O]': 7, '[=S]': 8, '[Branch]': 9, '[C]': 10, '[Cl]': 11, '[EOS]': 12, '[F]': 13, '[NH0+1]': 14, '[N]': 15, '[OH0-1]': 16, '[O]': 17, '[PAD]': 18, '[PH1]': 19, '[P]': 20, '[Ring1]': 21, '[Ring2]': 22, '[SOS]': 23, '[S]': 24, '[SiH0]': 25, '[pop]': 26}
27
tensor([ 2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([23,  2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([ 2, 10, 17, 10,  2, 12, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])


In [2]:
def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    

    return model, prior

In [3]:
choice = "Trans_MHA"
latent_dim = 128
model, prior = select_model(choice, latent_dim)

In [4]:
# -------------------------------------------------------------
# 1. 무작위 배치 가져오기
# -------------------------------------------------------------
batch = get_random_batch(val_dataloader)       # (*batch 형태에 맞게 수정*)
sm_enc, sm_dec_in, sm_dec_tgt, props = (t.to(device) for t in batch)

# -------------------------------------------------------------
# 2. IWAE
# -------------------------------------------------------------
if choice == "Trans" or choice == "Trans_MHA":
    iwae = iwae_bound(model, prior, sm_enc, sm_dec_in, sm_dec_tgt, props).mean().item()
elif choice == "LSTM":
    iwae = iwae_bound_lstm(model, prior, sm_dec_in, sm_dec_tgt, props).mean().item()
else:
    iwae = iwae_bound_lstm_MHA(model, prior, sm_dec_in, sm_dec_tgt, props).mean().item()
print(f"IWAE  : {iwae:.3f}")

IWAE  : -773.006
